In [1]:
import numpy as np

In [2]:
def generate_confs(orbinfo, active_space, options):
    nocc, nmo = orbinfo
    act_occ, act_vir = active_space
    occ_list = range(nocc-act_occ, nocc)
    vir_list = range(nocc, nocc+act_vir)
    csfs = [(0,0,0,0)]
    num_csfs = [1,0,0,0,0,0,0]
    if options['singles']:
        if options['full_cis']:
            D_ia = [(i,0,a,0) for i in range(nocc) for a in range(nocc,nmo)]
        else:    
            D_ia = [(i,0,a,0) for i in occ_list for a in vir_list]
        csfs.extend(D_ia)
        num_csfs[1] = len(D_ia)
    if options['doubles']:
        if options['doubles_iiaa']:
            D_iiaa = [(i,i,a,a) for i in occ_list for a in vir_list]
            csfs.extend(D_iiaa)
            num_csfs[2] = len(D_iiaa)
        if options['doubles_iiab']:      
            D_iiab = [(i,i,a,b) for i in occ_list for a in vir_list for b in vir_list if a>b]
            csfs.extend(D_iiab)
            num_csfs[3] = len(D_iiab)
        if options['doubles_ijaa']:
            D_ijaa = [(i,j,a,a) for i in occ_list for j in occ_list for a in vir_list if i>j]
            csfs.extend(D_ijaa)
            num_csfs[4] = len(D_ijaa)
        D_ijab = [(i,j,a,b) for i in occ_list for j in occ_list 
                            for a in vir_list for b in vir_list if i>j and a>b]
        if options['doubles_ijab_A']:
            D_ijab_A = D_ijab        
            csfs.extend(D_ijab_A)
            num_csfs[5] = len(D_ijab_A)
        if options['doubles_ijab_B']:        
            D_ijab_B = D_ijab
            csfs.extend(D_ijab_B)
            num_csfs[6] = len(D_ijab_B)
    return csfs, num_csfs

class CSF(object):
    """A class construct for configuration state functions
    """
    def __init__(self, csf_dict, spin=0):
        self.Dets = list(csf_dict.keys())
        self.coeff = list(csf_dict.values())
        self.Ns = 2*spin + 1

In [21]:
orbinfo = (8, 45)
active_space = (8, 37)
options = { 'singles' : True,
            'full_cis' : True,
            'doubles' : True,
            'doubles_iiaa' : True,
            'doubles_iiab' : True,
            'doubles_ijaa' : True,
            'doubles_ijab_A' : True,
            'doubles_ijab_B' : True}

confs, num_confs = generate_confs(orbinfo, active_space, options)
from pyci.configint.bitstrings import bitDet, SlaterCondon

def generate_csfs(orbinfo, confs, num_confs,options):
    csfs = []
    Q = 0
    nocc, nmo = orbinfo
    refDet = bitDet(alpha_orblist=range(nocc),beta_orblist=range(nocc)) 
    csfs.append(refDet)
    if options['singles']:
        n_ia = num_confs[1]
        for conf in confs[Q:Q+n_ia]:
            i,j,a,b = conf
            det1 = refDet.copy()
            det1.remove_alpha(i), det1.add_alpha(a)
            det2 = refDet.copy()
            det2.remove_beta(i), det2.add_beta(a)
            csfs.append(CSF({det1: 0.7071067811865475, 
                                det2: 0.7071067811865475}))
        Q += n_ia
    if options['doubles']:    
        n_iiaa = num_confs[2]
        n_iiab = num_confs[3]
        n_ijaa = num_confs[4]
        n_ijab_A = num_confs[5]
        n_ijab_B = num_confs[6]
        if options['doubles_iiaa']:
            for conf in confs[Q:Q+n_iiaa]:
                i,j,a,b = conf
                det1 = refDet.copy()
                det1.remove_alpha(i), det1.add_alpha(a)
                det1.remove_beta(i), det1.add_beta(a)
                csfs.append(CSF({det1 : 1.0}))
            Q += n_iiaa
        if options['doubles_iiab']:
            for conf in confs[Q:Q+n_iiab]:
                i,j,a,b = conf
                det1 = refDet.copy()
                det1.remove_alpha(i), det1.add_alpha(a)
                det1.remove_beta(i), det1.add_beta(b)
                det2 = refDet.copy()
                det2.remove_alpha(i), det1.add_alpha(b)
                det2.remove_beta(i), det1.add_beta(a)
                csfs.append(CSF({det1: 0.7071067811865475, 
                                    det2: 0.7071067811865475}))
            Q += n_iiab
        if options['doubles_ijaa']:
            for conf in confs[Q:Q+n_ijaa]:
                i,j,a,b = conf
                det1 = refDet.copy()
                det1.remove_alpha(i), det1.add_alpha(a)
                det1.remove_beta(j), det1.add_beta(a)
                det2 = refDet.copy()
                det2.remove_alpha(j), det1.add_alpha(a)
                det2.remove_beta(i), det1.add_beta(a)
                csfs.append(CSF({det1: 0.7071067811865475, 
                                    det2: 0.7071067811865475}))
            Q += n_ijaa
        if options['doubles_ijab_A']:
            for conf in confs[Q:Q+n_ijab_A]:
                i,j,a,b = conf
                det1 = refDet.copy()
                det1.remove_alpha(i), det1.add_alpha(a)
                det1.remove_alpha(j), det1.add_alpha(b)
                det2 = refDet.copy()
                det2.remove_beta(i), det1.add_beta(a)
                det2.remove_beta(j), det1.add_beta(b)
                det3 = refDet.copy()
                det3.remove_alpha(i), det1.add_alpha(a)
                det3.remove_beta(j), det1.add_beta(b)
                det4 = refDet.copy()
                det4.remove_alpha(j), det1.add_alpha(b)
                det4.remove_beta(i), det1.add_beta(a)
                det5 = refDet.copy()
                det5.remove_alpha(i), det1.add_alpha(b)
                det5.remove_beta(j), det1.add_beta(a)
                det6 = refDet.copy()
                det6.remove_alpha(j), det1.add_alpha(b)
                det6.remove_beta(i), det1.add_beta(a)
                csfs.append(CSF({det1: +0.5773502691896258, 
                                    det2: +0.5773502691896258,
                                    det3: +0.2886751345948129, 
                                    det4: +0.2886751345948129,
                                    det5: -0.2886751345948129, 
                                    det6: -0.2886751345948129}))
            Q += n_ijab_A
        if options['doubles_ijab_B']:
            for conf in confs[Q:Q+n_ijab_B]:
                i,j,a,b = conf
                det1 = refDet.copy()
                det1.remove_alpha(i), det1.add_alpha(a)
                det1.remove_beta(j), det1.add_beta(b)
                det2 = refDet.copy()
                det2.remove_alpha(i), det1.add_alpha(b)
                det2.remove_beta(j), det1.add_beta(a)
                det3 = refDet.copy()
                det3.remove_alpha(j), det1.add_alpha(a)
                det3.remove_beta(i), det1.add_beta(b)
                det4 = refDet.copy()
                det4.remove_alpha(j), det1.add_alpha(b)
                det4.remove_beta(i), det1.add_beta(a)
                csfs.append(CSF({det1: 0.50, 
                                    det2: 0.50,
                                    det3: 0.50, 
                                    det4: 0.50}))
            Q += n_ijab_B
    return csfs

In [22]:
csfs = generate_csfs(orbinfo, confs, num_confs,options)

In [23]:
class bitDet(object):
    """A class for Slater Determinants represented as Bit-strings 
    """
    def __init__(self, alpha_orblist=None, beta_orblist=None, alpha_bitstr=0, 
                beta_bitstr=0):
        """ Creates a Slater bitDets from lists of alpha and beta indices
        """
        if alpha_orblist != None and alpha_bitstr == 0:
            alpha_bitstr = bitDet.orblist2bitstr(alpha_orblist)
        if beta_orblist != None and beta_bitstr == 0:
            beta_bitstr = bitDet.orblist2bitstr(beta_orblist)     
        # Setting alpha, beta string attributes
        self.alpha_bitstr = alpha_bitstr
        self.beta_bitstr = beta_bitstr

    @staticmethod
    def orblist2bitstr(orblist):
        """Converts a list of MO indices to a bitstring
        """
        if len(orblist) == 0:
            return 0
        orblist = sorted(orblist, reverse=True)
        iPre = orblist[0]
        bitstr = 1
        for i in orblist:
            bitstr <<= iPre - i
            bitstr |= 1  # bitwise OR
            iPre = i
        bitstr <<= iPre  # LEFT-SHFIT by iPre bits
        return bitstr

    @staticmethod
    def bitstr2orblist(bitstr):
        """Converts a bitstring to a list of corresponding MO indices
        """
        i = 0 
        orblist = []
        while bitstr != 0:
            if bitstr & 1 == 1:  # bitwise AND
                orblist.append(i)
            bitstr >>= 1  # RIGHT-SHIFT by 1 bit
            i += 1
        return orblist

    @staticmethod
    def countbits(bitstr):
        """Counts number of orbitals in a bitstring
        """
        count = 0 
        while bitstr != 0:
            if bitstr & 1 == 1:
                count += 1
            bitstr >>= 1
        return count
    
    @staticmethod
    def orbpositions(bitstr, orblist): # redundant function
        """Returns positions of orbitals in a bit-determinant
        """
        count = 0 
        idx = 0
        positions = []
        for i in orblist:
            while idx < i:
                if bitstr & 1 == 1:
                    count += 1
                bitstr >>= 1
                idx += 1
            positions.append(count)
            continue
        return positions
    
    @staticmethod
    def countorbitals(orblist):
        return len(orblist)

    def __orbstr__(self):
        alist = bitDet.bitstr2orblist(self.alpha_bitstr)
        blist = bitDet.bitstr2orblist(self.beta_bitstr)
        return "|"+str(alist)+","+str(blist)+">"

    def __bitstr__(self):
        astr = bin(self.alpha_bitstr)
        bstr = bin(self.beta_bitstr)
        return "|"+astr+","+bstr+">"

    def copy(self):
        """ Deep copy of determinant object
        """
        return bitDet(alpha_bitstr=self.alpha_bitstr, beta_bitstr=self.beta_bitstr)

    # Creation and Annhilation operators for Alpha and Beta electrons
    def add_alpha(self, orbidx):
        """Adds an alpha electron(up-spin) into an MO with index=orbidx 
        """
        self.alpha_bitstr |= 1 << orbidx

    def add_beta(self, orbidx):
        """Adds an beta electron(down-spin) into an MO with index=orbidx
        """
        self.beta_bitstr |= 1 << orbidx

    def remove_alpha(self, orbidx):
        """Removes an alpha electron from an MO with index=orbidx
        """
        self.alpha_bitstr &= ~(1 << orbidx)

    def remove_beta(self, orbidx):
        """Removes an beta electron from an MO with index=orbidx
        """
        self.beta_bitstr &= ~(1 << orbidx)

class SlaterCondon:
    """ A sub-module implementing Slater-Condon rules
        explicitly in MO indices.
    """
    def __init__(self, orbinfo, mo_eps, mo_coeff, mo_erints):
        # currently only implemented for closed shell system
        nel, nbf, nmo = orbinfo
        nocc, nvir = int(nel/2), int((nmo-nel)/2)
        self.occ_list = range(nocc)
        self.mo_eps = mo_eps
        self.mo_coeff = mo_coeff
        self.mo_erints = mo_erints
        self.mo_fock_matrix = np.diag(mo_eps)
        
    @staticmethod
    def get_common_orblist(det1, det2):
        """Return a list of common orbitals
        """    
        alpha_common = bitDet.bitstr2orblist(det1.alpha_bitstr & det2.alpha_bitstr)
        beta_common = bitDet.bitstr2orblist(det1.beta_bitstr & det2.alpha_bitstr)
        return (alpha_common, beta_common)
    
    @staticmethod
    def get_diff_orblist(det1, det2):
        """Returns lists of unique [alpha, beta] orbitals in det1 and det2  
        """
        alpha_common = det1.alpha_bitstr & det2.alpha_bitstr
        beta_common = det1.beta_bitstr & det2.alpha_bitstr
        alpha_diff1 = bitDet.bitstr2orblist(det1.alpha_bitstr ^ alpha_common)
        alpha_diff2 = bitDet.bitstr2orblist(det2.alpha_bitstr ^ alpha_common)
        beta_diff1 = bitDet.bitstr2orblist(det1.beta_bitstr ^ beta_common)
        beta_diff2 = bitDet.bitstr2orblist(det2.beta_bitstr ^ beta_common)
        return (alpha_diff1, beta_diff1, alpha_diff2, beta_diff2)
    
    @staticmethod
    def num_diff_orbs(det1, det2):
        """Returns number of different alpha, beta orbitals b/w det1 & det2
        """
        num_diff_alpha = bitDet.countbits(det1.alpha_bitstr ^ det2.alpha_bitstr)
        num_diff_beta = bitDet.countbits(det1.beta_bitstr ^ det2.beta_bitstr)
        return (int(num_diff_alpha/2), int(num_diff_beta/2))        

    def one_elec_overlap(self, num_diff, common_orbs, diff_orbs, one_eprop):
        """Returns <det1 | O_{1} | det2 >
        """
        one_elec_overlap = 0 # if it differs by 2 or more 
        alpha_common, beta_common = common_orbs
        alpha_diff1, beta_diff1, alpha_diff2, beta_diff2 = diff_orbs
        if sum(num_diff) == 0:   # differ 0 orbs
            one_elec_overlap = 2*sum([self.mo_eps[i] for i in alpha_common])
        elif sum(num_diff) == 1: # differ 1 orbs
            if alpha_diff1 != [] and alpha_diff2 != []:
                m, p = alpha_diff1[0],alpha_diff2[0]
                one_elec_overlap = one_eprop[m,p]
            elif beta_diff1 != [] and beta_diff2 != []:
                one_elec_overlap = one_eprop[beta_diff1[0], beta_diff2[0]]  
        return one_elec_overlap

    def two_elec_overlap(self, num_diff, common_orbs, diff_orbs, two_eprop):
        """Returns < det1 | O_{2} | det2 >
        """
        two_elec_overlap = 0 
        alpha_common, beta_common = common_orbs
        alpha_diff1, beta_diff1, alpha_diff2, beta_diff2 = diff_orbs
        if sum(num_diff) == 0:   # differ 0 orbs
            two_elec_overlap = sum([2*two_eprop[a,b,a,b] - two_eprop[a,b,b,a] 
                                for a in alpha_common for b in alpha_common])
        elif sum(num_diff) == 1: # differ 1 orbs 
            if alpha_diff1 != [] and alpha_diff2 != []:
                m,p = alpha_diff1[0], alpha_diff2[0]
                two_elec_overlap = sum([2*two_eprop[m,a,p,a] - two_eprop[m,a,a,p]
                                        for a in alpha_common])                    
            elif beta_diff1 != [] and beta_diff2 != []:
                m,p = beta_diff1[0], beta_diff2[0]
                two_elec_overlap = sum([2*two_eprop[m,a,p,a] - two_eprop[m,a,a,p]
                                        for a in beta_common])                    
        elif sum(num_diff) == 2: # differ 2 orbs
            pass
            # needs a loop to sort spins to get matrix elements
            # if alpha_diff1==[] and alpha_diff2==[]:
            #     m,n = beta_diff1
            #     p,q = beta_diff2
            #     two_elec_overlap = two_eprop[m,n,p,q] - two_eprop[m,n,q,p]  
            # elif beta_diff1==[] and beta_diff2==[]:
            #     m,n = alpha_diff1
            #     p,q = alpha_diff2  
            #     two_elec_overlap = two_eprop[m,n,p,q] - two_eprop[m,n,q,p]  
            # elif alpha_diff1==[] and alpha_diff2!=[] or beta_diff1==[] and beta_diff2!=[]:
            #     two_elec_overlap = 0
            # else:
            #     two_elec_overlap = 0
        return two_elec_overlap

    def comp_hmatrix_elem(self, det1, det2):
        """Returns matrix element < det1 | H | det2 >
        """
        h_elem = 0 
        num_diff = SlaterCondon.num_diff_orbs(det1, det2)
        if int(sum(num_diff)) in [0,1,2]:
            common_orbs = SlaterCondon.get_common_orblist(det1, det2)
            diff_orbs = SlaterCondon.get_diff_orblist(det1,det2)
            h_elem += self.one_elec_overlap(num_diff, common_orbs, diff_orbs, self.mo_fock_matrix) 
            h_elem += self.two_elec_overlap(num_diff, common_orbs, diff_orbs, self.mo_erints) 
        return h_elem


44253